<img src="https://www.e4ds.com/news_photo/U77C53G6CP8ASEHUJ5B7.png">

In [1]:
import torch.nn as nn
import pandas as pd
import torch
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

True

In [2]:
train_df = pd.read_csv("data/asl_data/sign_mnist_train.csv")
valid_df = pd.read_csv("data/asl_data/sign_mnist_valid.csv")

In [3]:
train_df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,3,107,118,127,134,139,143,146,150,153,...,207,207,207,207,206,206,206,204,203,202
1,6,155,157,156,156,156,157,156,158,158,...,69,149,128,87,94,163,175,103,135,149
2,2,187,188,188,187,187,186,187,188,187,...,202,201,200,199,198,199,198,195,194,195
3,2,211,211,212,212,211,210,211,210,210,...,235,234,233,231,230,226,225,222,229,163
4,12,164,167,170,172,176,179,180,184,185,...,92,105,105,108,133,163,157,163,164,179


In [ ]:
sample_df = train_df.head().copy() # 행 5개만 sample_df로 copy
sample_df.pop('label')
sample_x = sample_df.values
sample_x # 각 픽셀의 값이 0 ~ 255 범위인 것 확인 

array([[107, 118, 127, ..., 204, 203, 202],
       [155, 157, 156, ..., 103, 135, 149],
       [187, 188, 188, ..., 195, 194, 195],
       [211, 211, 212, ..., 222, 229, 163],
       [164, 167, 170, ..., 163, 164, 179]], dtype=int64)

In [6]:
sample_x.shape # 현재 Flatten 되어 있는 상태 

(5, 784)

In [8]:
IMG_HEIGHT = 28
IMG_WIDTH = 28
IMG_CHS = 1

sample_x = sample_x.reshape(-1, IMG_CHS, IMG_HEIGHT, IMG_WIDTH)
sample_x.shape # turn into batch

(5, 1, 28, 28)

In [9]:
class MyDataset(Dataset):
    def __init__(self, base_df):
        x_df = base_df.copy()  # Some operations below are in-place
        y_df = x_df.pop('label')
        x_df = x_df.values / 255  # Normalize values from 0 to 1
        x_df = x_df.reshape(-1, IMG_CHS, IMG_HEIGHT, IMG_WIDTH)
        self.xs = torch.tensor(x_df).float().to(device)
        self.ys = torch.tensor(y_df).to(device)

    def __getitem__(self, idx):
        x = self.xs[idx]
        y = self.ys[idx]
        return x, y

    def __len__(self):
        return len(self.xs)

In [10]:
BATCH_SIZE = 32

train_data = MyDataset(train_df)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE)
train_N = len(train_loader.dataset)

valid_data = MyDataset(valid_df)
valid_loader = DataLoader(valid_data, batch_size=BATCH_SIZE)
valid_N = len(valid_loader.dataset)

### Convolutional Model

In [21]:
n_classes = 24
kernel_size = 3
flattened_img_size = 75 * 3 * 3 # Flatten 차원 정의 

model = nn.Sequential(
    # First convolution
    nn.Conv2d(IMG_CHS, 25, kernel_size, stride=1, padding=1),  # 25 x 28 x 28
    nn.BatchNorm2d(25),
    nn.ReLU(),
    nn.MaxPool2d(2, stride=2),  # 25 x 14 x 14
    # Second convolution
    nn.Conv2d(25, 50, kernel_size, stride=1, padding=1),  # 50 x 14 x 14
    nn.BatchNorm2d(50),
    nn.ReLU(),
    nn.Dropout(.2),
    nn.MaxPool2d(2, stride=2),  # 50 x 7 x 7
    # Third convolution
    nn.Conv2d(50, 75, kernel_size, stride=1, padding=1),  # 75 x 7 x 7
    nn.BatchNorm2d(75),
    nn.ReLU(),
    nn.MaxPool2d(2, stride=2),  # 75 x 3 x 3
    # Flatten to Dense
    nn.Flatten(),
    nn.Linear(flattened_img_size, 512),
    nn.Dropout(.3),
    nn.ReLU(),
    nn.Linear(512, n_classes)
).to(device)

In [22]:
loss_function = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters())

In [23]:
def get_batch_accuracy(output, y, N):
    pred = output.argmax(dim=1, keepdim=True)
    correct = pred.eq(y.view_as(pred)).sum().item()
    return correct / N

In [24]:
def validate():
    loss = 0
    accuracy = 0

    model.eval()
    with torch.no_grad():
        for x, y in valid_loader:
            output = model(x)

            loss += loss_function(output, y).item()
            accuracy += get_batch_accuracy(output, y, valid_N)
    print('FIXME - Loss: {:.4f} Accuracy: {:.4f}'.format(loss, accuracy))

In [25]:
def train():
    loss = 0
    accuracy = 0

    model.train()
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        output = model(x)
        optimizer.zero_grad()
        batch_loss = loss_function(output, y)
        batch_loss.backward()
        optimizer.step()

        loss += batch_loss.item()
        accuracy += get_batch_accuracy(output, y, train_N)
    print('FIXME - Loss: {:.4f} Accuracy: {:.4f}'.format(loss, accuracy))

In [26]:
epochs = 20

for epoch in range(epochs):
    print('Epoch: {}'.format(epoch))
    train()
    validate()

Epoch: 0
FIXME - Loss: 259.5551 Accuracy: 0.9129
FIXME - Loss: 18.3103 Accuracy: 0.9682
Epoch: 1
FIXME - Loss: 18.3847 Accuracy: 0.9944
FIXME - Loss: 25.8760 Accuracy: 0.9612
Epoch: 2
FIXME - Loss: 14.3183 Accuracy: 0.9958
FIXME - Loss: 22.3777 Accuracy: 0.9727
Epoch: 3
FIXME - Loss: 14.2531 Accuracy: 0.9947
FIXME - Loss: 46.8148 Accuracy: 0.9304
Epoch: 4
FIXME - Loss: 3.8916 Accuracy: 0.9988
FIXME - Loss: 11.0379 Accuracy: 0.9858
Epoch: 5
FIXME - Loss: 0.2773 Accuracy: 1.0000
FIXME - Loss: 9.5492 Accuracy: 0.9852
Epoch: 6
FIXME - Loss: 19.8459 Accuracy: 0.9933
FIXME - Loss: 19.8100 Accuracy: 0.9749
Epoch: 7
FIXME - Loss: 0.6651 Accuracy: 0.9999
FIXME - Loss: 17.5414 Accuracy: 0.9777
Epoch: 8
FIXME - Loss: 11.1702 Accuracy: 0.9958
FIXME - Loss: 11.6419 Accuracy: 0.9834
Epoch: 9
FIXME - Loss: 1.3981 Accuracy: 0.9997
FIXME - Loss: 13.7200 Accuracy: 0.9763
Epoch: 10
FIXME - Loss: 9.4013 Accuracy: 0.9968
FIXME - Loss: 24.4539 Accuracy: 0.9644
Epoch: 11
FIXME - Loss: 6.6615 Accuracy: 0.9979